In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

In [30]:
# Make Dataset
d = 1
L = 6

n = 10000
x_raw = np.random.random_sample((n,d)) * L
print(x_raw)

y_raw = (np.max(x_raw % 1, axis=1) < 1 / np.power(2, 1/d) ).astype(int)
print(sum(y_raw))

p_train = 0.8
x_train = x_raw[:int(n * p_train)]
y_train = y_raw[:int(n * p_train)]
x_test = x_raw[int(n * p_train):]
y_test = y_raw[int(n * p_train):]

[[ 5.47494858]
 [ 1.29964151]
 [ 5.36956077]
 ..., 
 [ 4.49046515]
 [ 5.03594853]
 [ 3.14299441]]
4995


In [82]:
# Build models and set up loss function and optimizer

dtype = torch.FloatTensor

simple_model = nn.Sequential(
                nn.Linear(d, 300),
                nn.ReLU(inplace=True),
                nn.Linear(300, 2),
              )

simple_model.type(dtype)

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(simple_model.parameters(), lr=1e-2)

In [95]:
batch_size = 1000

num_batches = int(len(x_train) / batch_size)

num_rounds = 5000

rand_label = [np.random.randint(num_batches) for i in range(num_rounds)]

train_batches = [(i, (x_train[rand_label[i] * batch_size: (rand_label[i] + 1) * batch_size],
                      y_train[rand_label[i] * batch_size: (rand_label[i] + 1) * batch_size]))
                      for i in range(num_rounds)]

In [96]:
print_every = 1000

for t, (x, y) in train_batches:
    x_var = Variable(torch.from_numpy(x).type(dtype))
    y_var = Variable(torch.from_numpy(y).type(dtype).long())

    # This is the forward pass: predict the scores for each class, for each x in the batch.
    scores = simple_model(x_var)
    
    # Use the correct y values and the predicted y values to compute the loss.
    loss = loss_fn(scores, y_var)
    
    if (t + 1) % print_every == 0:
        print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

    # Zero out all of the gradients for the variables which the optimizer will update.
    optimizer.zero_grad()
    
    # This is the backwards pass: compute the gradient of the loss with respect to each 
    # parameter of the model.
    loss.backward()
    
    # Actually update the parameters of the model using the gradients computed by the backwards pass.
    optimizer.step()

t = 1000, loss = 0.0739
t = 2000, loss = 0.0582
t = 3000, loss = 0.0597
t = 4000, loss = 0.0524
t = 5000, loss = 0.0488


In [97]:
x_var = Variable(torch.from_numpy(x_test).type(dtype), volatile=True)
scores = simple_model(x_var)
_, preds = scores.data.cpu().max(1)
correct_rate = (preds == torch.from_numpy(y_test)).sum() / len(x_test)
print(correct_rate)

0.977
